In [ ]:
#Connecting to an API/Pulling in the Data and Cleaning/Formatting
#Perform at least 5 data transformation and/or cleansing steps to your API data. The below examples are not required - they are just potential transformations you could do. If your data doesn't work for these scenarios, complete different transformations. You can do the same transformation multiple times if you needed to clean your data. The goal is a clean dataset at the end of the milestone.
#Replace Headers
#Format data into a more readable format
#Identify outliers and bad data
#Find duplicates
#Fix casing or inconsistent values
#Conduct Fuzzy Matching

In [2]:
# load packages
import pandas as pd
import datetime as dt
import pandas_profiling as pp
from fuzzywuzzy import fuzz
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi

/Users/dragon/opt/anaconda3/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
mkdir ~/.kaggle

mkdir: /Users/dragon/.kaggle: File exists


In [4]:
# API Data Pull 1
api = KaggleApi()
api.authenticate()

In [5]:
api.dataset_download_file('kamilpytlak/personal-key-indicators-of-heart-disease', file_name='heart_2020_cleaned.csv')

True

In [18]:
#API Data Pull 2
import requests
import json
response_API = requests.get('https://heartapi.herokuapp.com/predict?age=31&sex=1&cigs=5&chol=230&sBP=280&dia=0&dBP=90&gluc=87&hRate=84')

In [19]:
data = response_API.text

In [20]:
json.loads(data)

{'data': {'age': '31',
  'cigsPerDay': '5',
  'diaBP': '90',
  'diabetes': '0',
  'glucose': '87',
  'heartRate': '84',
  'sex': '1',
  'sysBP': '280',
  'totChol': '230'},
 'prediction': [1],
 'probability': [[0.4587093009776524, 0.5412906990223476]]}

In [22]:
import zipfile
with zipfile.ZipFile("heart_2020_cleaned.csv.zip","r") as zip_ref:
    zip_ref.extractall()

In [24]:
# Printing Data
heart_API_data = pd.read_csv('heart_2020_cleaned.csv') 
heart_API_data_df=pd.DataFrame(heart_API_data)
heart_API_data_df

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,3.0,30.0,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes
1,No,20.34,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No
2,No,26.58,Yes,No,No,20.0,30.0,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No
3,No,24.21,No,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes
4,No,23.71,No,No,No,28.0,0.0,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319790,Yes,27.41,Yes,No,No,7.0,0.0,Yes,Male,60-64,Hispanic,Yes,No,Fair,6.0,Yes,No,No
319791,No,29.84,Yes,No,No,0.0,0.0,No,Male,35-39,Hispanic,No,Yes,Very good,5.0,Yes,No,No
319792,No,24.24,No,No,No,0.0,0.0,No,Female,45-49,Hispanic,No,Yes,Good,6.0,No,No,No
319793,No,32.81,No,No,No,0.0,0.0,No,Female,25-29,Hispanic,No,No,Good,12.0,No,No,No


In [25]:
# 1 - Renaming Column
heart_API_data_df.rename(columns={'HeartDisease': 'Heart_Disease'}, inplace=True)
heart_API_data_df

,Heart_Disease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,3.0,30.0,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes
1,No,20.34,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No
2,No,26.58,Yes,No,No,20.0,30.0,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No
3,No,24.21,No,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes
4,No,23.71,No,No,No,28.0,0.0,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319790,Yes,27.41,Yes,No,No,7.0,0.0,Yes,Male,60-64,Hispanic,Yes,No,Fair,6.0,Yes,No,No
319791,No,29.84,Yes,No,No,0.0,0.0,No,Male,35-39,Hispanic,No,Yes,Very good,5.0,Yes,No,No
319792,No,24.24,No,No,No,0.0,0.0,No,Female,45-49,Hispanic,No,Yes,Good,6.0,No,No,No
319793,No,32.81,No,No,No,0.0,0.0,No,Female,25-29,Hispanic,No,No,Good,12.0,No,No,No


In [26]:
#2 - Dropping Duplicates

len_1 = heart_API_data_df.shape[0]
heart_API_data_df_dedup = heart_API_data_df.drop_duplicates()
len_2 = heart_API_data_df_dedup.shape[0]
print(len_1, len_2)

319795 301717


In [27]:
#3 - Dataframe with no duplicate for further analysis
#heart_API_data_dedup_df = pd.DataFrame(heart_API_data_df_dedup)

pp.ProfileReport(heart_API_data_df_dedup)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In the above data, row# 1 and 3 both has same Smoking habbit. Hence, they should be a 100% match

In [30]:
#4 - Fuzzy Matching
fuzz.partial_ratio(heart_API_data_df_dedup['Smoking'].values[1], heart_API_data_df_dedup['Smoking'].values[3])

100

In [31]:
#5 - Counting
heart_API_data_df_dedup.AgeCategory.value_counts()

65-69          31670
60-64          31219
70-74          29273
55-59          27610
50-54          23736
80 or older    23352
75-79          20713
45-49          20518
18-24          19998
40-44          19837
35-39          19526
30-34          17953
25-29          16312
Name: AgeCategory, dtype: int64